通过 scikit-learn 库提供的便捷工具生成 2000 个线性不可分的2分类数据集

数据集可直接使用 scikit-learn 提供的 make_moons 函数生成

In [34]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import matplotlib.cm as cm #colormaps
import numpy as np

In [35]:
# 采样点数
N_SAMPLES = 2000
# 测试比率,7:3 划分训练和测试
TEST_SIZE = 0.3
x, y = make_moons(n_samples = N_SAMPLES, noise=0.2, random_state=100)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, random_state=42)

In [36]:
# 可视化
def make_plot(x, y, plot_name, xx=None, yy=None, preds=None, dark=False):
    """
    x:两列数据
    y:数据的颜色标记
    plot_name:图表名称
    darks:图表的背景样式
    """
    if(dark):
        plt.style.use('dark_background')
    else:
        sns.set_style('whitegrid')
    plt.figure(figsize=(16, 12))
    axes = plt.gca()
    axes.set(xlabel="$x_1$", ylabel="$x_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.2)
    plt.subplots_adjust(right=0.8)
    if(xx is not None and yy is not None and preds is not None):
        # 填充等高线
        plt.contourf(xx, yy, preds.reshape(xx.shape), 25, alpha=1, cmap=cm.Spectral)
        # 等高线图
        plt.contour(xx, yy, preds.reshape(xx.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    # 绘制散点图
    # s:散点的大小，c:色彩或颜色序列，ravel 返回扁平化数组
    plt.scatter(x[:, 0], x[:, 1], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='none')
    # plt.savefig('datasets.svg')
    plt.close()

新建类 Layer 实现一个网络层

In [37]:
class Layer:
    def __init__(self, n_input, n_neurons, activation=None, weights=None, bias=None):
        """
        参数：
        n_input -- 输入节点数
        n_neurous -- 输出节点数
        activation -- 激活函数
        weights -- 权重张量，默认内部生成
        bias -- 偏置，同上
        """
        # 初始化权重和偏置
        self.weights = weights if weights is not None else np.random.randn(n_input, n_neurons) * np.sqrt(1 / n_neurons)
        self.bias = bias if bias is not None else np.random.randn(n_neurons) * 0.1
        self.activation = activation
        self.last_activation = None
        self.error = None #记录当前层 delta 的中间变量
        self.delta = None
    # 前向传播
    def activate(self, x):
        r = np.dot(x, self.weights) + self.bias
        # last_activation 保存当前层的输出值
        self.last_activation =self._apply_activation(r)
        return self.last_activation
    # 激活函数设置
    def _apply_activation(self, r):
        if self.activation is None:
            return r
        elif self.activation == "relu":
            return np.maximum(r, 0)
        elif self.activation == "tanh":
            return np.tanh(r)
        elif self.activation == "sigmoid":
            return 1 / (1 + np.exp(-r))
    # 计算激活函数的导数
    def apply_activation_derivative(self, r):
        if self.activation is None:
            return np.ones_like(r)
        elif self.activation == "relu":
            grad = np.array(r, copy=True)
            grad[r > 0] = 1.
            grad[r <= 0] = 0.
            return grad
        elif self.activation == "tanh":
            return 1 - r ** 2
        elif self.activation == "sigmoid":
            return r * (1 - r)
        return r

In [38]:
# 实现网络模型的 NeuralNetwork类
class NerualNetwork:
    def __init__(self):
        # 网络层对象列表
        self._layers = []
    def add_layer(self, layer):
        self._layers.append(layer)
    def feed_forward(self, x):
        for layer in self._layers:
            x = layer.activate(x)
        return x
    # 网络模型的反向传播实现
    def backpropagation(self, x, y, learning_rate):
        output = self.feed_forward(x)
        for i in reversed(range(len(self._layers))):
            layer = self._layers[i]
            if layer == self._layers[-1]:
                layer.error = y - output
                layer.delta = layer.error * layer.apply_activation_derivatve(
                    output)
            else:
                next_layer = self._layers[i + 1]
                layer.error = np.dot(next_layer.weights, next_layer.delta)
                layer.delta = layer.error * layer.apply_activation_derivate(
                    layer.last_activation)
        # 更新权值
        for i in range(len(self._layers)):
            layer = self._layers[i]
            o_i = np.atleast_2d(x if i == 0 else self._layers[i -
                                                            1].last_activation)
            layer.weights += layer.delta * o_i.T * learning_rate
    def train(self, x_train, x_test, y_train, y_test, learning_rate, max_epochs):
        y_onehot = np.zeros((y_train.shape[0], 2))
        y_onehot[np.arange(y_train.shape[0], y_train)] = 1

        mses = []
        for i in range(max_epochs):
            for j in range(len(x_train)):
                self.backpropagation(x_train[j], y_onehot[j], learning_rate)
            if i % 10 == 0:
                mse = np.mean(np.square(y_onehot - self.feed_forward(x_train)))
                mses.append(mse)
                print('Epochs:%s,MSE %f' % (i, float(mse)))
                # 统计并打印准确率
                print("Accuracy:%.2f %%" % (self.accuracy(self.predict(x_test),y_test.flatten()) * 100))
        return mses

In [ ]:
model = NerualNetwork()
model.add_layer(Layer(2, 25, 'sigmoid'))
model.add_layer(Layer(25, 50, 'sigmoid'))
model.add_layer(Layer(50, 25, 'sigmoid'))
model.add_layer(Layer(25, 2, 'sigmoid'))